In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

# titanic data

Najpierw zaloguj się do https://www.kaggle.com/ i przejdź do wyzwania https://www.kaggle.com/c/titanic, aby pobrać 
 * train.csv i test.csv. 

Zapisz je w katalogu datasets/titanic.

In [150]:
import os
TITANIC_PATH = os.path.join("datasets", "titanic")

In [151]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [152]:
from sklearn.model_selection import train_test_split

seed = 42069
data = load_titanic_data("train.csv")
#train_data, test_data = train_test_split(data, train_size=0.8, test_size=0.2, random_state=seed)
X_test = load_titanic_data("test.csv")
#y_test = load_titanic_data("gender_submission.csv")
train_data = data

* Dane są już podzielone na zestaw treningowy i zestaw testów. 
* Jednak dane testowe nie zawierają etykiet: Twoim celem jest wyszkolenie najlepszego modelu, który możesz wykorzystać w danych treningowych, następnie dokonanie swoich przewidywań na danych testowych i przesłanie ich do Kaggle, aby zobaczyć ostateczny wynik.

Rzućmy okiem na kilka pierwszych rzędów zestawu treningowego:

In [153]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The attributes have the following meaning:

* Survived: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
* Pclass: passenger class.
* Name, Sex, Age: self-explanatory
* SibSp: how many siblings & spouses of the passenger aboard the Titanic.
* Parch: how many children & parents of the passenger aboard the Titanic.
* Ticket: ticket id
* Fare: price paid (in pounds)
* Cabin: passenger's cabin number
* Embarked: where the passenger embarked the Titanic
* Let's get more info to see how much data is missing:

In [154]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Atrybuty **Age**, **Cabin** oraz **Embarked** są czasami zerowe (mniej niż 891 wartości bez wartości null), szczególnie w przypadku **Cabin** (77% ma wartość zerową). Zignorujemy teraz **Cabin** i skupimy się na reszcie. Atrybut **Age** ma około 19% wartości pustych, więc będziemy musieli zdecydować, co z nimi zrobić. Zastąpienie wartości null medianą wieku wydaje się uzasadnione.

Atrybuty **Name** i **Ticket** mogą mieć pewną wartość, ale będą one nieco trudne do przekształcenia w użyteczne liczby. Na razie będziemy je ignorować.

Rzućmy okiem na atrybuty liczbowe:

In [155]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* Tylko 38% przeżyło: to wystarczająco blisko do 40%, więc **accuracy** będzie rozsądną miarą do oceny naszego modelu.

Sprawdźmy, czy etykiety przyjmują wartości 0 lub 1:

In [156]:
X_train = train_data.drop(labels="Survived", axis=1)
y_train = train_data["Survived"].values
np.unique(y_train)

array([0, 1], dtype=int64)

Nie zapomnij o etykietach:

In [157]:
from sklearn.preprocessing import LabelEncoder

y_train = LabelEncoder().fit_transform(y_train)

Teraz rzućmy okiem na wszystkie atrybuty kategoryczne:

In [158]:
train_data["Name"].value_counts()

Braund, Mr. Owen Harris                     1
Boulos, Mr. Hanna                           1
Frolicher-Stehli, Mr. Maxmillian            1
Gilinski, Mr. Eliezer                       1
Murdlin, Mr. Joseph                         1
                                           ..
Kelly, Miss. Anna Katherine "Annie Kate"    1
McCoy, Mr. Bernard                          1
Johnson, Mr. William Cahoone Jr             1
Keane, Miss. Nora A                         1
Dooley, Mr. Patrick                         1
Name: Name, Length: 891, dtype: int64

In [159]:
train_data["Ticket"].value_counts()

347082      7
CA. 2343    7
1601        7
3101295     6
CA 2144     6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: Ticket, Length: 681, dtype: int64

In [160]:
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

Atrybut **Embarked** mówi nam, gdzie pasażer zaokrętował: C = Cherbourg, Q = Queenstown, S = Southampton.

Teraz zbudujmy nasze **pipeline** preprocessingu. 

Wykorzystamy DataframeSelector aby wybrać określone atrybuty z DataFrame:

In [161]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

Zbudujmy **pipeline** dla atrybutów numerycznych:

In [162]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["PassengerId", "Age", "SibSp", "Parch", "Fare", "Pclass"])),
        ("scaler", MinMaxScaler()),
        ("imputer", SimpleImputer(strategy="median"))
    ])

In [163]:
num_pipeline.fit_transform(X_train)

array([[0.        , 0.27117366, 0.125     , 0.        , 0.01415106,
        1.        ],
       [0.0011236 , 0.4722292 , 0.125     , 0.        , 0.13913574,
        0.        ],
       [0.00224719, 0.32143755, 0.        , 0.        , 0.01546857,
        1.        ],
       ...,
       [0.99775281, 0.34656949, 0.125     , 0.33333333, 0.04577135,
        1.        ],
       [0.9988764 , 0.32143755, 0.        , 0.        , 0.0585561 ,
        0.        ],
       [1.        , 0.39683338, 0.        , 0.        , 0.01512699,
        1.        ]])

Będziemy także potrzebować imputera do kategorycznych kolumn  napisowych (zwykły Imputer nie działa na tych kolumnach):

In [164]:
# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

Teraz możemy zbudować **pipeline** dla atrybutów kategorycznych:

In [165]:
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["Ticket", "Name", "Sex", "Embarked"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore')),
    ])

In [166]:
cat_pipeline.fit_transform(X_train)

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

Na koniec połączmy powyższe podejścia:

In [167]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

Teraz mamy fajny **pipeline** przetwarzania wstępnego, który pobiera dane wejściowe i zwraca dane wyjściowe złorzone z liczb, które możemy podać do dowolnego modelu uczenia maszynowego.

In [168]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.ensemble import VotingClassifier


kfold = StratifiedKFold(n_splits=10)

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('classifier', SVC(kernel='linear'))])


param_grid = {
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
}

grid_1 = GridSearchCV(pipe, param_grid, cv=kfold, scoring="accuracy")

grid_1.fit(X_train, y_train)
print(grid_1.best_params_)

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('classifier', SVC(kernel='rbf'))])

param_grid = {
            'classifier__gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_2 = GridSearchCV(pipe, param_grid, cv=kfold, scoring="accuracy")

grid_2.fit(X_train, y_train)
print(grid_2.best_params_)

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', SVC(kernel='poly'))])
param_grid = {
            'classifier__degree': [1, 2, 3, 4, 5],
            'classifier__coef0': [0, 1],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_3 = GridSearchCV(pipe, param_grid, cv=kfold, scoring="accuracy")

grid_3.fit(X_train, y_train)
print(grid_3.best_params_)


pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('classifier', LogisticRegression(max_iter=10000))])

param_grid = {
            'classifier__C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
            'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'],
}
grid_4 = GridSearchCV(pipe, param_grid, cv=kfold, scoring="accuracy") #scoring, czy lepszy
grid_4.fit(X_train, y_train)
print(grid_4.best_params_)

# pipe = Pipeline([
#     ('preprocessing', preprocess_pipeline),
#     ('classifier', LogisticRegression())])

# param_grid = {
#             'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
#             'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'],
# }
# grid_5 = RandomizedSearchCV(pipe, param_grid, cv=kfold)
# grid_5.fit(X_train, y_train)
# grid_5.best_params_

{'classifier__C': 1}
{'classifier__C': 10, 'classifier__gamma': 0.1}
{'classifier__C': 1, 'classifier__coef0': 1, 'classifier__degree': 3}


f:\Anaconda\AnacondaFiles\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "f:\Anaconda\AnacondaFiles\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "f:\Anaconda\AnacondaFiles\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "f:\Anaconda\AnacondaFiles\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "f:\Anaconda\AnacondaFiles\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties,

{'classifier__C': 10000, 'classifier__penalty': 'l2'}


# Zad

Robimy StratifiedKFold i znajdujemy optymalne parametry dla

* SVM z jądrem rbf
* SVM z jądrem poly
* SVM liniowego
* Regresji logistycznej

In [169]:
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

models = []
models.append(('SVM linear', grid_1.best_estimator_))
models.append(('SVM rbf', grid_2.best_estimator_))
models.append(('SVM poly', grid_3.best_estimator_))
models.append(('Logistic regression', grid_4.best_estimator_))
#models.append(('Randomized logistic regression', grid_5.best_estimator_))
# print(X_train.dtypes)

# lbl = preprocessing.LabelEncoder()
# X_train['Name'] = lbl.fit_transform(X_train['Name'].astype(float))
# X_train['Sex'] = lbl.fit_transform(X_train['Sex'].astype(float))
# X_train['Ticket'] = lbl.fit_transform(X_train['Ticket'].astype(float))
# X_train['Embarked'] = lbl.fit_transform(X_train['Embarked'].astype(float))


# xgb_cl = xgb.XGBClassifier()
# xgb_cl.fit(X_train, y_train)

# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [3, 4, 5]
#         }

# folds = 3
# param_comb = 5

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# random_search = RandomizedSearchCV(xgb_cl, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3)
# random_search.fit(X_train, y_train)

# models.append(("XGBoost", random_search.best_estimator_))
# print("precision_score: {}".format(metrics.precision_score(y_test, random_search.best_estimator_.predict(X_test)) ))

vc = VotingClassifier(models)
vc.fit(X_train, y_train)

models.append(('Voting classifier', vc))

precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test, model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test, model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test, model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test, model.predict(X_test)) ))
    precision_score.append(metrics.precision_score(y_test, model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test, model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test, model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test, model.predict(X_test)))

SVM linear


ValueError: Found input variables with inconsistent numbers of samples: [179, 418]

In [170]:
import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['SVM linear', 'SVM rbf', 'SVM poly', 'Logistic regression', 'Voting Classifier'])
df

,Method,precision_score,recall_score,f1_score,accuracy_score
0,SVM linear,NaN,NaN,NaN,NaN
1,SVM rbf,NaN,NaN,NaN,NaN
2,SVM poly,NaN,NaN,NaN,NaN
3,Logistic regression,NaN,NaN,NaN,NaN
4,Voting Classifier,NaN,NaN,NaN,NaN


In [171]:
X_pred = load_titanic_data("test.csv")
prediction = grid_3.best_estimator_.predict(X_pred)

dict_variable = {value[0]:prediction[index] for (index,value) in X_pred.iterrows()}

pd.DataFrame(dict_variable.items(), columns=['PassengerId', 'Survived']).to_csv('prediction.csv', index=False)